In [24]:
from dotenv import load_dotenv
import os
load_dotenv('../.env_api')
from rich.pretty import pprint
from operator import add

def ppprint(obj):
    pprint(obj, indent_guides=False)

os.environ["HTTP_PROXY"] = "http://localhost:9090"
os.environ["HTTPS_PROXY"] = "http://localhost:9090"
os.environ["REQUESTS_CA_BUNDLE"] = "/Users/tomek/Library/Application Support/com.proxyman.NSProxy-setapp/app-data/proxyman-ca.pem"
os.environ["SSL_CERT_FILE"] = "/Users/tomek/Library/Application Support/com.proxyman.NSProxy-setapp/app-data/proxyman-ca.pem"
from langsmith import Client
import os

from typing import List, Annotated, Any, Dict, Optional
from pydantic import BaseModel, Field

import instructor
from openai import OpenAI

import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))


from api.rag.utils.utils import lc_messages_to_regular_messages
from api.rag.utils.utils import prompt_template_config
from api.core.config import config

### Coordinator agent

In [25]:
class MCPToolCall(BaseModel):
    name: str
    arguments: dict
    server: str

class ToolCall(BaseModel):
    name: str
    arguments: dict

class RAGUsedContext(BaseModel):
    id: str
    description: str
    
class Delegation(BaseModel):
    agent: str
    task: str = Field(default="")

class CoordinatorAgentResponse(BaseModel):
    next_agent: str
    plan: list[Delegation]
    final_answer: bool = Field(default=False)
    answer: str

class State(BaseModel):
    messages: Annotated[List[Any], add] = []
    answer: str = ""

    coordinator_iteration: int = Field(default=0)
    product_qa_iteration: int = Field(default=0)
    shopping_cart_iteration: int = Field(default=0)

    coordinator_final_answer: bool = Field(default=False)
    product_qa_final_answer: bool = Field(default=False)
    shopping_cart_final_answer: bool = Field(default=False)

    product_qa_available_tools: List[Dict[str, Any]] = []
    shopping_cart_available_tools: List[Dict[str, Any]] = []

    tool_calls: Optional[List[ToolCall]] = Field(default_factory=list)
    mcp_tool_calls: Optional[List[MCPToolCall]] = Field(default_factory=list)
    retrieved_context: List[RAGUsedContext] = Field(default_factory=list)
    
    user_id: str = ""
    cart_id: str = ""

    next_agent: str = ""
    plan: list[Delegation] = Field(default_factory=list)

    trace_id: str = ""

In [26]:
def coordinator_agent_node(state) -> dict:
    template = prompt_template_config('../' + config.PROMPT_TEMPLATE_PATH, 'coordinator_agent')
    
    prompt = template.render()

    messages = state.messages

    conversation = []

    for msg in messages:
        conversation.append(lc_messages_to_regular_messages(msg))

    client = instructor.from_openai(OpenAI(api_key=config.OPENAI_API_KEY))

    response, raw_response = client.chat.completions.create_with_completion(
            model="gpt-4.1",
            response_model=CoordinatorAgentResponse,
            messages=[{"role": "system", "content": prompt}, *conversation],
            temperature=0,
    )


    return {
        # "messages": ai_message,
        "next_agent": response.next_agent,
        "plan": response.plan,
        "coordinator_final_answer": response.final_answer,
        "coordinator_iteration": state.coordinator_iteration + 1,
        "answer": response.answer,
        "trace_id": ''
    }

In [30]:
initial_state = State(messages=[    
    {"role": "user", "content": "What is the weather in Tokyo?"}
])

answer = coordinator_agent_node(initial_state)

In [31]:
pprint(answer)

{
│   'next_agent': '',
│   'plan': [],
│   'coordinator_final_answer': True,
│   'coordinator_iteration': 1,
│   'answer': 'I am a shopping assistant and do not have access to weather information. If you have any questions about products or shopping, please let me know how I can assist you!',
│   'trace_id': ''
}

#### Create coordinator evaluation dataset

In [32]:
coordinator_eval_dataset = [
    {
        'inputs': {
            'messages': [
                {
                    'role': 'user',
                    'content': 'What is the weather today?'
                }
            ]
        },
        'outputs': {
            'next_agent': 'product_qa_agent',
            'coordinator_final_answer': True
        }
    },  
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can I get some earphones?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add an item with ID B09NLTDHQ6 to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add those earphones to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add the best items to my cart? I am looking for laptop bags."}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you find some good reviews for items in my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "shopping_cart_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you put the items with the most positive user reviews to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "What kind of stuff do you sell?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you help me with my order?"}
            ]
        },
        "outputs": {
            "next_agent": "",
            "coordinator_final_answer": True
        }
    },
    {
        "inputs": {
            "messages": [
                {"role": "user", "content": "Can you add two, ideally red tablets to my cart?"}
            ]
        },
        "outputs": {
            "next_agent": "product_qa_agent",
            "coordinator_final_answer": False
        }
    }
]

### Upload the dataset

In [38]:
ls_client = Client(api_key=os.getenv("LANGSMITH_API_KEY"))



In [ ]:
dataset_name = "coordinator-evaluation-dataset"
dataset = ls_client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating routing of the coordinator agent",
)


In [ ]:
for item in coordinator_eval_dataset:
    ls_client.create_example(
        dataset_id=dataset.id,
        inputs=item['inputs'],
        outputs=item['outputs']
    )

In [39]:
def next_agent_evaluator(run, example):
    next_agent_match = run.outputs['next_agent'] == example.outputs['next_agent']
    final_answer_match = run.outputs['coordinator_final_answer'] == example.outputs['coordinator_final_answer']

    return all([next_agent_match, final_answer_match])

In [40]:
results = ls_client.evaluate(
    lambda x: coordinator_agent_node(State(messages=x['messages'])),
    data="coordinator-evaluation-dataset",
    evaluators = [
        next_agent_evaluator,
    ],
    experiment_prefix='coordinator-evaluation-dataset'
)

/Users/tomek/dev/ai-bootcamp/01-ai-engineering-bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'coordinator-evaluation-dataset-ebba6756' at:
https://smith.langchain.com/o/c5906f8c-2b0e-445e-8ba7-ea63d7432c8c/datasets/a8a58390-b576-4b77-8875-b5c42217c390/compare?selectedSessions=ef509e60-8c4c-4426-b0d1-64ab33151d55




10it [00:24,  2.43s/it]
